# Deep: NLP With Transformer -  GPT

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df_copom = pd.read_csv('df_copom_label.csv')

In [3]:
df_copom.head()

,Date,Selic,Meeting_Number,Decision,Decision_txt,label_hawk_dove,label_next_meet,Text,Type
0,2006/03/08,16.50,117.0,-0.75,decrease,dovish,decrease,"In the March Meeting, the Banco Central do Br...",Statement
1,2006/04/19,15.75,118.0,-0.75,decrease,dovish,decrease,"In the April Meeting, the Monetary Policy Com...",Statement
2,2006/05/31,15.25,119.0,-0.50,decrease,dovish,decrease,"In the May Meeting, the Monetary Policy Commi...",Statement
3,2006/07/19,14.75,120.0,-0.50,decrease,dovish,decrease,"In the July Meeting, the Copom unanimously de...",Statement
4,2006/08/30,14.25,121.0,-0.50,decrease,dovish,decrease,"In the August Meeting, the Copom unanimously ...",Statement


In [4]:
df_copom.iloc[29]

Date                                                      2009/10/21
Selic                                                           8.75
Meeting_Number                                                   146
Decision                                                           0
Decision_txt                                                 mantain
label_hawk_dove                                              neutral
label_next_meet                                              mantain
Text                  \tBrasília - In light of inflation prospect...
Type                                                       Statement
Name: 29, dtype: object

In [5]:
df_copom.shape

(159, 9)

### Split test and train

In [6]:
from sklearn.model_selection import train_test_split

C:\Users\ftrav\anaconda3\envs\pytorchenv\lib\site-packages\sklearn\utils\validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'


In [7]:
X = df_copom.copy()
y = df_copom['label_hawk_dove']

In [8]:
#Perform train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [9]:
texts = X_train['Text'].tolist()
labels = y_train.tolist()

In [10]:
len(texts)

143

In [11]:
len(labels)

143

In [12]:
seq_len = 512 # number of the tokens tokenizer will create
num_samples = len (texts)
num_samples, seq_len

(143, 512)

#### Convert labels to one-hot encoded vectors

In [13]:
label_classes = list(set(labels))
num_classes = len(label_classes)

label_to_index = {label: index for index, label in enumerate(label_classes)}
index_to_label = {index: label for label, index in label_to_index.items()}

labels_encoded = np.array([label_to_index[label] for label in labels])
labels = np.eye(num_classes)[labels_encoded]
labels[:5]

array([[0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

In [14]:
len(labels)

143

## ChatCPG

In [15]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW

In [16]:
# Define the BERT model
model_name = 'bert-base-uncased'
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)
tokenizer = BertTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [17]:
# Set up the device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [18]:
# Custom dataset class
class CentralBankDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        return text, label

In [19]:
# Create the dataset
dataset = CentralBankDataset(texts, labels)

In [20]:
# Tokenize the texts and create data loader
def collate_fn(batch):
    texts, labels = zip(*batch)
    inputs = tokenizer.batch_encode_plus(
        texts,
        padding=True,
        truncation=True,
        return_tensors="pt"
    )
    return inputs, torch.tensor(labels)

batch_size = 2
dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=collate_fn)

In [21]:
# Training configuration
epochs = 5
learning_rate = 2e-5

In [22]:
# Set optimizer and scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
total_steps = len(dataloader) * epochs
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=total_steps)

In [ ]:
# Training loop
model.train()
for epoch in range(epochs):
    total_loss = 0
    for i, (inputs, labels) in enumerate(dataloader):
        inputs = {key: val.to(device) for key, val in inputs.items()}
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(**inputs, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{epochs}, Average Loss: {avg_loss:.4f}")

C:\Users\ftrav\anaconda3\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:204.)
  # Remove the CWD from sys.path while we load stuff.


Epoch 1/5, Average Loss: 0.6516
Epoch 2/5, Average Loss: 0.6131
Epoch 3/5, Average Loss: 0.5453


In [ ]:
# Testing
model.eval()

In [ ]:
len(X_test['Text'])

In [98]:
with torch.no_grad():
    # for i, text in enumerate(test_texts):
    for i, row in X_test.iterrows():
        inputs = tokenizer.encode_plus(
            row['Text']    ,
            padding='longest',
            truncation=True,
            max_length=512,
            return_tensors="pt"
        ).to(device)
        outputs = model(**inputs)
        print(np.argmax(outputs.logits[0].cpu().numpy()))
        X_test.at[i, 'Sentiment'] = np.argmax(outputs.logits[0].cpu().numpy())

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [94]:
X_test

,Date,Selic,Meeting_Number,Decision,Decision_txt,label_hawk_dove,label_next_meet,Text,Type,Sentiment
78,2016/03/02,14.25,197.0,0.00,mantain,neutral,mantain,The Copom released the following note to the ...,Statement,0.0
155,2023/03/22,13.75,253.0,0.00,mantain,neutral,mantain,A) Update of economic outlook and Copom’s scen...,Minutes,0.0
128,2021/06/16,4.25,239.0,0.75,increase,hawkish,increase,A) Update of economic outlook and Copom’s base...,Minutes,0.0
55,2013/04/17,7.50,174.0,0.25,increase,hawkish,increase,The Copom released the following note to the ...,Statement,0.0
94,2018/03/21,6.50,213.0,-0.25,decrease,dovish,mantain,The Copom unanimously decided to reduce the Se...,Statement,0.0
29,2009/10/21,8.75,146.0,0.00,mantain,neutral,mantain,\tBrasília - In light of inflation prospect...,Statement,0.0
147,2022/09/21,13.75,249.0,0.00,mantain,dovish,mantain,A) Update of economic outlook and Copom’s scen...,Minutes,0.0
51,2012/10/10,7.25,170.0,-0.25,decrease,dovish,mantain,The Copom released the following note to the ...,Statement,0.0
98,2018/09/19,6.50,217.0,0.00,mantain,neutral,mantain,​The Copom unanimously decided to maintain the...,Statement,0.0
141,2022/05/04,12.75,246.0,1.00,increase,hawkish,increase,"​In its 246th meeting, the Copom unanimously d...",Statement,0.0
